# DNN-Based DDoS Anomaly Detection in the Network Data Plane

In this 4-part notebook series, we show how a quantized neural network (QNN) can be trained to classify packets as belonging to DDoS (malicious) or regular (benign) network traffic flows. The model is trained with quantized weights and activations, and we use the [Brevitas](https://github.com/Xilinx/brevitas) framework to train the QNN. The model is then converted into an FPGA-friendly RTL implementation for high-throughput inference, which can be integrated with a packet-processing pipeline in the network data plane.

This notebook series is composed of 4 parts. Below is a brief summary of what each part covers.

[Part 1](./1-train.ipynb): How to use Brevitas to train a quantized neural network for our target application, which is classifying packets as belonging to malicious/DDoS or benign/normal network traffic flows. The output trained model at the end of this part is a pure software implementation, i.e. it cannot be converted to a custom RTL FINN model to run on an FPGA just yet.

[Part 2](./2-prepare.ipynb): This notebook focuses on taking the output software model from the previous part and preparing it for hardware-friendly implementation using the FINN framework. The notebook describes the steps taken to "surgery" the software model in order for hardware generation via FINN. We also verify that all the changes made to the software model in this notebook DO NOT affect the output predictions in the "surgeried" model.

[Part 3](./3-build.ipynb): In this notebook, we use the FINN framework to build the custom RTL accelerator for our target model. FINN can generate a variety of RTL accelerators, and this notebook covers some build configuration parameters that influence these outputs.

[Part 4](./4-verify.ipynb): The generated hardware is simulated using cycle-accurate RTL simulation tools, and its outputs are compared against the original software-only model trained in part one. The output model from this step is now ready to be integrated into a larger FPGA design, which in this context is a packet-processing network data plane pipeline designed for identifying anomalous DDoS flows from benign flows.

This tutorial series is a supplement to our demo paper presented at EuroP4 2023 workshop, titled [Enabling DNN Inference in the Network Data Plane](https://dl.acm.org/doi/10.1145/3630047.3630191). You can cite our work using the following BibTeX snippet:

```
@inproceedings{siddhartha2023enabling,
  title={Enabling DNN Inference in the Network Data Plane},
  author={Siddhartha and Tan, Justin and Bansal, Rajesh and Chee Cheun, Huang and Tokusashi, Yuta and Yew Kwan, Chong and Javaid, Haris and Baldi, Mario},
  booktitle={Proceedings of the 6th on European P4 Workshop},
  pages={65--68},
  year={2023}
}
```

# Part 4: Verifying generated FINN hardware accelerator

First step, house-keeping!

### House-keeping

In [19]:
import os
import onnx
import json
import random
from os.path import join
from copy import deepcopy
from collections import Counter
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from brevitas.nn import QuantLinear, QuantReLU, QuantIdentity
from finn.util.visualization import showInNetron
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from qonnx.transformation.general import GiveUniqueNodeNames
from qonnx.custom_op.registry import getCustomOp
import finn.core.onnx_exec as oxe

from utils.common import bcolors
from utils.dataset import CICIDS2017_PerPacket
from utils.train_test import test, verify_onnx

# set seed for reproducibility
random.seed(0)

# Path to this end-to-end example's directory
EXAMPLE_DIR = join(os.environ['FINN_ROOT'], "notebooks/end2end_example/ddos-anomaly-detector")

# Path to build directory from part one
BUILD_DIR_P1 = join(EXAMPLE_DIR, "build", "part_01")
BUILD_DIR_P2 = join(EXAMPLE_DIR, "build", "part_02")
BUILD_DIR_P3 = join(EXAMPLE_DIR, "build", "part_03")

# Path to build directory to write outputs from this notebook to
BUILD_DIR = join(EXAMPLE_DIR, "build", "part_04")
os.makedirs(BUILD_DIR, exist_ok=True)

# Path to where datasets are stored
DATASET_DIR = join(EXAMPLE_DIR, "data")

# get the target device to run model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Target device: {device}")

Target device: cpu


Let's also load (i) binarized test set and (ii) software trained Brevitas model from Part 1 of this notebook series.

In [20]:
# (i) Load the binarized test set

# load the dataset_metadata.json config parameters
with open(join(BUILD_DIR_P1, "dataset_metadata.json"), "r") as fp:
    dataset_metadata = json.load(fp)

# extract feature names from metadata
feature_columns = [x[0] for x in dataset_metadata["ordering"]]

test_set_fpath = join(DATASET_DIR, "cicids2017-split.test.csv")
dataset = CICIDS2017_PerPacket(test_set_fpath)
test_set, _ = dataset.get_binarized_dataset(feature_columns)

# Batch size to use for inference
batch_size = 1000
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

Loading CIC-IDS2017 per-packet-level dataset
Loaded dataset of length = 130777
Dataset statistics: 67375/130777 (51.52% TRUE labels)
Dataset metadata: {
    "total_in_bitwidth": 128,
    "ordering": [
        [
            "total_bytes",
            32
        ],
        [
            "duration_usec",
            64
        ],
        [
            "total_pkts",
            16
        ],
        [
            "total_urg",
            16
        ]
    ],
    "total_out_bitwidth": 1
}


In [21]:
# (ii) Load the software trained Brevitas model

# Declare model architecture -- note that this must be identical to the model used
# in part one, or else there may be errors loading in the trained weights
input_size = dataset_metadata["total_in_bitwidth"]
hidden1 = 32
hidden2 = 32
weight_bit_width = 2
act_bit_width = 2
num_classes = dataset_metadata["total_out_bitwidth"]

brevitas_model = nn.Sequential(
      QuantLinear(input_size, hidden1, bias=True, weight_bit_width=weight_bit_width),
      nn.BatchNorm1d(hidden1),
      nn.Dropout(0.5),
      QuantReLU(bit_width=act_bit_width),
      QuantLinear(hidden1, hidden2, bias=True, weight_bit_width=weight_bit_width),
      nn.BatchNorm1d(hidden2),
      nn.Dropout(0.5),
      QuantReLU(bit_width=act_bit_width),
      QuantLinear(hidden2, num_classes, bias=True, weight_bit_width=weight_bit_width)
)

# Make sure the model is on CPU before loading a pretrained state_dict
brevitas_model = brevitas_model.cpu()

# Load pretrained weights
trained_state_dict = torch.load(join(BUILD_DIR_P1, "trained_model.pth"))
brevitas_model.load_state_dict(trained_state_dict, strict=True)

# Move model to target device and run inference on test set
brevitas_model.to(device)
print(f"Trained Brevitas model test accuracy = {100*test(brevitas_model, test_loader, device):.4f}%")

Trained Brevitas model test accuracy = 85.1426%


### RTL simulation of stitched IP

Next, we do an RTL simulation of the stitched IP that has been created in part 3. Note that we can also do a CPP simualation of the HLS layers, which is something not covered in this notebook, but details about it can be found [in this notebook](https://github.com/Xilinx/finn/blob/v0.10/notebooks/end2end_example/bnn-pynq/tfc_end2end_verification.ipynb). A CPP simulation is a behavioural simulation, which can be used to debug intermediate HLS code that was generated by the FINN compiler.

In order to do an RTL simulation, FINN docker comes with PyVerilator, which is a python-wrapper for Verilator, an open-source C++ based verilog simulator. We use the split ONNX graph mode to carry out the RTL simulation, where a parent dataflow graph consists of non-HW layers, while the child graph consists of all HW layers. The child model is referenced in the parent model using the `model` attribute in the `StreamingDataflowPartition` node in the parent graph.

Let's start by first loading the child model, which consists of all the HLS/RTL stitched IP layers. We take the ONNX graph of the step that creates the stitched IP.

In [22]:
model_stitched_ip_fpath = join(BUILD_DIR_P3, "output_rtl", "intermediate_models", "step_create_stitched_ip.onnx")
child_model_for_rtlsim = ModelWrapper(model_stitched_ip_fpath)

Next, we compile and create the Verilator emulation files, which prepares the model for RTL simulation.

In [15]:
%%capture

test_fpga_part = "xcu250-figd2104-2L-e"
target_clk_ns = 4.0

child_model_for_rtlsim = child_model_for_rtlsim.transform(GiveUniqueNodeNames())
child_model_for_rtlsim = child_model_for_rtlsim.transform(PrepareIP(test_fpga_part, target_clk_ns))
child_model_for_rtlsim = child_model_for_rtlsim.transform(HLSSynthIP())
child_model_for_rtlsim = child_model_for_rtlsim.transform(SetExecMode("rtlsim"))
child_model_for_rtlsim = child_model_for_rtlsim.transform(PrepareRTLSim())

We can save the child model and view it in Netron.

In [16]:
output_rtlsim_dir = join(BUILD_DIR, "output_rtlsim")
os.makedirs(output_rtlsim_dir, exist_ok=True)

child_model_for_rtlsim_fpath = join(output_rtlsim_dir, "child-model-for-rtlsim.onnx")
child_model_for_rtlsim.save(child_model_for_rtlsim_fpath)
showInNetron(child_model_for_rtlsim_fpath)

The next part is to embed the child model into the parent model, which is also generated in the `intermediate_models` directory. The `StreamingDataflowPartition` node (`sdp_node`) is annotated with the `model` property that points to the child model that was saved in the previous cell. The metadata property `exec_mode` is set to "rtlsim" for the whole child model, which means the entire child model is executed as a single stitched IP simulation.

In [17]:
parent_model_fpath = join(BUILD_DIR_P3, "output_rtl", "intermediate_models", "dataflow_parent.onnx")
parent_model = ModelWrapper(parent_model_fpath)
sdp_node = getCustomOp(parent_model.graph.node[0])
sdp_node.set_nodeattr("model", child_model_for_rtlsim_fpath)
parent_model = parent_model.transform(SetExecMode("rtlsim"))

Now we are ready to run RTL simulation of the whole stitched IP! Note that this will be slower than software simulation, so we verify with a fewer number of examples. We can re-use our `verify_onnx()` helper function, since we are using the ONNX interface to do the RTL simulation using PyVerilator in the backend. Note that this cell may produce a lot of output (typically Verilator warnings), so you may have to scroll down to see the result. You should see all predictions match with the software model (i.e. `nok=0` in the progress bar), and non-zero counts for `tp` and `tn`, which will build confidence that the RTL model is predicting correctly on both positive/negative classes.

In [61]:
num_verif = 10
verif_tensors = test_set.tensors[0][:num_verif]
print(f"Running verification with tensor:\n{verif_tensors}")
if verify_onnx(parent_model, brevitas_model, verif_tensors, device):
    print(f"{bcolors.OKGREEN}Stitched IP model output matches with reference Brevitas software model output!{bcolors.ENDC}")
else:
    print(f"{bcolors.FAIL}Stitched IP model output differs from reference Brevitas software model output! Something went wrong...{bcolors.ENDC}")

Running verification with tensor:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


Verification (ONNX):   0%|                                                    | 0/10 [00:00<?, ?it/s]%Warning-TIMESCALEMOD: /home/sids/workspace/project_find_ml/finn/finn-rtllib/swg/swg_pkg.sv:32:9: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   32 | package swg;
      |         ^~~
                       /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:539:8: ... Location of module with timescale
  539 | module MVAU_hls_0_Matrix_Vector_Activate_Batch (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       ... For warning description see https://verilator.org/warn/TIMESCALEMOD?v=4.224
                       ... Use "/* verilator lint_off TIMESCALEMOD */" and lint_on around source to disable this message.
%Warning-TIMESCALEMOD: /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:61:8: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   61 | module finn_

make: Entering directory '/tmp/finn_dev_sids/pyverilator_ipstitched_mdl_nlmq'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /tmp/finn_dev_sids/pyverilator_ipstitched_mdl_nlmq/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/local/share/

%Warning-TIMESCALEMOD: /home/sids/workspace/project_find_ml/finn/finn-rtllib/swg/swg_pkg.sv:32:9: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   32 | package swg;
      |         ^~~
                       /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:539:8: ... Location of module with timescale
  539 | module MVAU_hls_0_Matrix_Vector_Activate_Batch (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       ... For warning description see https://verilator.org/warn/TIMESCALEMOD?v=4.224
                       ... Use "/* verilator lint_off TIMESCALEMOD */" and lint_on around source to disable this message.
%Warning-TIMESCALEMOD: /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:61:8: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   61 | module finn_design_StreamingFIFO_rtl_2_0 (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                     

make: Entering directory '/tmp/finn_dev_sids/pyverilator_ipstitched_e2fe9k77'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /tmp/finn_dev_sids/pyverilator_ipstitched_e2fe9k77/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/local/share/

...

make: Entering directory '/tmp/finn_dev_sids/pyverilator_ipstitched_swln399v'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /tmp/finn_dev_sids/pyverilator_ipstitched_swln399v/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/local/share/

%Warning-TIMESCALEMOD: /home/sids/workspace/project_find_ml/finn/finn-rtllib/swg/swg_pkg.sv:32:9: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   32 | package swg;
      |         ^~~
                       /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:539:8: ... Location of module with timescale
  539 | module MVAU_hls_0_Matrix_Vector_Activate_Batch (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       ... For warning description see https://verilator.org/warn/TIMESCALEMOD?v=4.224
                       ... Use "/* verilator lint_off TIMESCALEMOD */" and lint_on around source to disable this message.
%Warning-TIMESCALEMOD: /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:61:8: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   61 | module finn_design_StreamingFIFO_rtl_2_0 (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                     

make: Entering directory '/tmp/finn_dev_sids/pyverilator_ipstitched_bk_krk8l'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /tmp/finn_dev_sids/pyverilator_ipstitched_bk_krk8l/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/local/share/

%Warning-TIMESCALEMOD: /home/sids/workspace/project_find_ml/finn/finn-rtllib/swg/swg_pkg.sv:32:9: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   32 | package swg;
      |         ^~~
                       /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:539:8: ... Location of module with timescale
  539 | module MVAU_hls_0_Matrix_Vector_Activate_Batch (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       ... For warning description see https://verilator.org/warn/TIMESCALEMOD?v=4.224
                       ... Use "/* verilator lint_off TIMESCALEMOD */" and lint_on around source to disable this message.
%Warning-TIMESCALEMOD: /tmp/finn_dev_sids/vivado_stitch_proj__pfun89u/finn_design_wrapper.v:61:8: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
   61 | module finn_design_StreamingFIFO_rtl_2_0 (
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                     

make: Entering directory '/tmp/finn_dev_sids/pyverilator_ipstitched__qvqa6dq'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /tmp/finn_dev_sids/pyverilator_ipstitched__qvqa6dq/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/local/share/

**Note** that input data is packed in `LSB` to `MSB` order. For example, if our input is an `8 x 1b` vector, and its value is `[1, 1, 1, 1, 0, 0, 0, 0]`, then the `[7:0]` input bus will be driven with the following stimulus: `8'b00001111`. All of this data-packing and unpacking is handled by the PyVerilator backend provided by the FINN framework.

And that's it! In the future, we will show how to integrate this FINN model as an inference engine into our larger FPGA-based SmartNIC platform.